In [1]:
!pip install sqlalchemy==1.3.9
!pip3 install ipython-sql

  Using cached SQLAlchemy-1.3.9-cp39-cp39-macosx_10_9_x86_64.whl
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.9
    Uninstalling SQLAlchemy-2.0.9:
      Successfully uninstalled SQLAlchemy-2.0.9
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.3.9 which is incompatible.
  Using cached SQLAlchemy-2.0.9-cp39-cp39-macosx_10_9_x86_64.whl (2.0 MB)
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.3.9
    Uninstalling SQLAlchemy-1.3.9:
      Successfully uninstalled SQLAlchemy-1.3.9


In [2]:
%load_ext sql

In [3]:
import csv, sqlite3

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [4]:
!pip install -q pandas==1.1.5

In [5]:
%sql sqlite:///my_data1.db

In [32]:
import pandas as pd
import datetime
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
for i in range(df.shape[0]):
    df.loc[i,'Date']=datetime.datetime.strptime(str(df.loc[i,'Date']),'%d-%m-%Y').strftime('%Y-%m-%d')
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

In [7]:
%sql select distinct "Launch_Site" from SPACEXTBL

 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


In [8]:
%sql select * from SPACEXTBL where "Launch_Site" like 'CCA%' limit 5

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing _Outcome
2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2012-05-22,07:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
2012-10-08,00:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


In [9]:
%sql select SUM(PAYLOAD_MASS__KG_) from SPACEXTBL where "Customer"=="NASA (CRS)"

 * sqlite:///my_data1.db
Done.


SUM(PAYLOAD_MASS__KG_)
45596


In [10]:
%sql select AVG(PAYLOAD_MASS__KG_) from SPACEXTBL where "Booster_Version"=="F9 v1.1"

 * sqlite:///my_data1.db
Done.


AVG(PAYLOAD_MASS__KG_)
2928.4


In [11]:
%sql select MIN("Date") from SPACEXTBL where "Landing _Outcome"=="Success (ground pad)" 

 * sqlite:///my_data1.db
Done.


"MIN(""Date"")"
2015-12-22


In [12]:
%sql select "Booster_Version" from SPACEXTBL where "Landing _Outcome"=="Success (drone ship)"\
and "PAYLOAD_MASS__KG_">4000 and "PAYLOAD_MASS__KG_"<6000

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 FT B1022
F9 FT B1026
F9 FT B1021.2
F9 FT B1031.2


In [13]:
%sql select count(*) from SPACEXTBL where "Mission_Outcome"=="Success"

 * sqlite:///my_data1.db
Done.


count(*)
98


In [14]:
%sql select count(*) from SPACEXTBL where "Mission_Outcome" like 'Failure%'

 * sqlite:///my_data1.db
Done.


count(*)
1


In [15]:
%sql select "Booster_Version" from SPACEXTBL where\
"PAYLOAD_MASS__KG_"=(select max(PAYLOAD_MASS__KG_) from SPACEXTBL)

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6


In [46]:
%sql select substr(Date, 6, 2) as "Month", "Landing _Outcome", "Booster_Version", "Launch_Site" from SPACEXTBL\
where "Landing _Outcome"="Failure (drone ship)" and substr(Date,1,4)='2015'

 * sqlite:///my_data1.db
Done.


Month,Landing _Outcome,Booster_Version,Launch_Site
01,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
04,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40


In [47]:
%sql select "Landing _Outcome" as "Landing outcome", COUNT("Landing _Outcome")\
as "Number of successful landings" from SPACEXTBL\
where "Date" between '2010-06-04' and '2017-03-20'\
group by "Landing _Outcome" order by "Number of successful landings" DESC

 * sqlite:///my_data1.db
Done.


Landing outcome,Number of successful landings
No attempt,10
Success (drone ship),5
Failure (drone ship),5
Success (ground pad),3
Controlled (ocean),3
Uncontrolled (ocean),2
Failure (parachute),2
Precluded (drone ship),1
